# Dataset Download and Creation of Train, Val, Test sets

In [ ]:
import os
import requests
import zipfile

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

def download_data(data_path):
    toy_data_path = os.path.join(data_path, 'fever_data.zip')
    toy_data_url_id = "1wArZhF9_SHW17WKNGeLmX-QTYw9Zscl1"
    toy_url = "https://docs.google.com/uc?export=download"

    if not os.path.exists(data_path):
        os.makedirs(data_path)

    if not os.path.exists(toy_data_path):
        print("Downloading FEVER data splits...")
        with requests.Session() as current_session:
            response = current_session.get(toy_url,
                                   params={'id': toy_data_url_id},
                                   stream=True)
        save_response_content(response, toy_data_path)
        print("Download completed!")

        print("Extracting dataset...")
        with zipfile.ZipFile(toy_data_path) as loaded_zip:
            loaded_zip.extractall(data_path)
        print("Extraction completed!")

download_data('dataset')

Download completed!
Extracting dataset...
Extraction completed!


In [ ]:
# Import pandas 
import pandas as pd 
import numpy as np
# reading our datasets
train_df = pd.read_csv("dataset/train_pairs.csv",index_col=0) 
val_df = pd.read_csv("dataset/val_pairs.csv",index_col=0) 
test_df = pd.read_csv("dataset/test_pairs.csv",index_col=0) 

train_df.head()

,Claim,Evidence,ID,Label
0,Chris Hemsworth appeared in A Perfect Getaway.,2\tHemsworth has also appeared in the science ...,3,SUPPORTS
1,Roald Dahl is a writer.,0\tRoald Dahl -LRB- -LSB- langpronˈroʊ.əld _ ˈ...,7,SUPPORTS
2,Roald Dahl is a governor.,0\tRoald Dahl -LRB- -LSB- langpronˈroʊ.əld _ ˈ...,8,REFUTES
3,Ireland has relatively low-lying mountains.,10\tThe island 's geography comprises relative...,9,SUPPORTS
4,Ireland does not have relatively low-lying mou...,10\tThe island 's geography comprises relative...,10,REFUTES


In [ ]:
train_df['Evidence'][6]

'16\tAfter suppressing a minor rebellion in Wales in 1276 -- 77 , Edward responded to a second rebellion in 1282 -- 83 with a full-scale war of conquest .\tfull-scale war of conquest\tConquest of Wales by Edward I'

# Text preprocessing

In [ ]:
# Let's do some data cleaning on the Evidence field of our data set
import re
from functools import reduce
import nltk
from nltk.corpus import stopwords

# Config
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
GOOD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
try:
    STOPWORDS = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    STOPWORDS = set(stopwords.words('english'))
    
def lower(text):
    """
    Transforms given text to lower case.
    Example:
    Input: 'I really like New York city'
    Output: 'i really like new your city'
    """

    return text.lower()

def replace_special_characters(text):
    """
    Replaces special characters, such as paranthesis,
    with spacing character
    """

    return REPLACE_BY_SPACE_RE.sub(' ', text)

def filter_out_uncommon_symbols(text):
    """
    Removes any special character that is not in the
    good symbols list (check regular expression)
    """

    return GOOD_SYMBOLS_RE.sub('', text)

def remove_stopwords(text):
    return ' '.join([x for x in text.split() if x and x not in STOPWORDS])

def remove_extra_spaces(text):
    """
    Removes any left or right spacing (including carriage return) from text.
    Example:
    Input: '  This assignment is cool\n'
    Output: 'This assignment is cool'
    """

    return re.sub('\s+',' ',text)

def strip_text(text):
    """
    Removes any left or right spacing (including carriage return) from text.
    Example:
    Input: '  This assignment is cool\n'
    Output: 'This assignment is cool'
    """

    return text.strip()

def remove_starting_index(text):
    """
    Removes: pronunciation description from the text, it comes in between -LSB- and -RSB
    Input: '2\tHemsworth has also appeared in the science'
    Output: 'Hemsworth has also appeared in the science'
    """

    return re.sub('^[0-9]+\t', '', text)


def remove_pronunciation(text):
    """
    Removes: pronunciation description from the text, it comes in between -LSB- and -RSB
    Input: 'Vietnam -LRB- ˌ ; -LSB- vîət nāːm -RSB- -RRB- , officially the Socialist Republic of Vietnam -LRB- \n'
    Output: 'Vietnam -LRB-ˌ ; -RRB- , officially the Socialist Republic of Vietnam\n'
    """

    return re.sub('-LSB- .* -RSB- ', '', text)

NLTK_SYM = re.compile('\-[A-Z]{3}\-')

def remove_NLTK_sym(text):
    """
    Removes: NLTK special tokens, eg. LRB (Left Round Bracket) and RRB (Right Round Brackets) .
    Example:
    Input: 'Vietnam -LRB- ˌ ; -LSB- vîət nāːm -RSB- -RRB- , officially the Socialist Republic of Vietnam -LRB- \n'
    Output: 'Vietnam ˌ ; vîət nāːm , officially the Socialist Republic of Vietnam\n'
    """

    return NLTK_SYM.sub('', text)

def remove_tabs(text):
    """
    Removes: NLTK special tokens, eg. LRB (Left Round Bracket) and RRB (Right Round Brackets) .
    Example:
    Input: 'Vietnam -LRB- ˌ ; -LSB- vîət nāːm -RSB- -RRB- , officially the Socialist Republic of Vietnam -LRB- \n'
    Output: 'Vietnam ˌ ; vîət nāːm , officially the Socialist Republic of Vietnam\n'
    """

    return re.sub('\t',' ', text)

def label_parse(label):
    label = label.to_numpy()
    for i in range(len(label)):
        label[i] = 1 if label[i] == "SUPPORTS" else 0
    return pd.DataFrame(label)
    
PREPROCESSING_PIPELINE = [
                          remove_starting_index,
                          remove_pronunciation,
                          remove_NLTK_sym,
                          lower,
                          replace_special_characters,
                          remove_extra_spaces,
                          remove_tabs,
                          strip_text
                          ]

# Anchor method

def text_prepare(text, filter_methods=None):
    """
    Applies a list of pre-processing functions in sequence (reduce).
    Note that the order is important here!
    """

    filter_methods = filter_methods if filter_methods is not None else PREPROCESSING_PIPELINE

    return reduce(lambda txt, f: f(txt), filter_methods, text)

# Pre-processing

print('Pre-processing evidence...')
# Replace each sentence with its pre-processed version
train_df['Evidence']=train_df['Evidence'].apply(lambda txt: text_prepare(txt))
val_df['Evidence']=val_df['Evidence'].apply(lambda txt: text_prepare(txt))
test_df['Evidence']=test_df['Evidence'].apply(lambda txt: text_prepare(txt))

# Converting claims into lower case
train_df['Claim']=train_df['Claim'].str.strip().str.lower()
val_df['Claim']=val_df['Claim'].str.strip().str.lower()
test_df['Claim']=test_df['Claim'].str.strip().str.lower()

train_df['Label'] = label_parse(train_df['Label'])
test_df['Label'] = label_parse(test_df['Label'])
val_df['Label'] = label_parse(val_df['Label'])


print("Pre-processing completed!")
train_df.head(10)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Pre-processing evidence...
Pre-processing completed!


,Claim,Evidence,ID,Label
0,chris hemsworth appeared in a perfect getaway.,hemsworth has also appeared in the science fic...,3,1
1,roald dahl is a writer.,roald dahl 13 september 1916 -- 23 november 19...,7,1
2,roald dahl is a governor.,roald dahl 13 september 1916 -- 23 november 19...,8,0
3,ireland has relatively low-lying mountains.,the island 's geography comprises relatively l...,9,1
4,ireland does not have relatively low-lying mou...,the island 's geography comprises relatively l...,10,0
5,there have been many notable performances by d...,his most commercially successful role to date ...,14,1
6,edward i of england responded to a second rebe...,after suppressing a minor rebellion in wales i...,17,1
7,h. h. holmes owned a building west of chicago.,many victims were said to have been killed in ...,19,1
8,h. h. holmes was the owner of a building locat...,many victims were said to have been killed in ...,20,1
9,the beastie boys released paul's boutique.,in 2009 they released digitally remastered del...,21,1


In [ ]:
train_df['Evidence'][1]

'roald dahl 13 september 1916 -- 23 november 1990 was a british novelist short story writer poet screenwriter and fighter pilot . fighter pilot fighter pilot'

# Dataset preprocessing

In [ ]:
#%%time
from nltk import sent_tokenize
nltk.download('punkt')

def xtr_evidence(df):
    """
    From a dataframe with the format [Claim, Evidence1.Evidence2..., Label] 
    Returns a list of the form [Claim, Evidence1,Label], [Claim, Evidence2, Label]...
    """
    ev_list=[]
    ids = [] # the i-th element of ids is the id of the i-th triple [Claim,Evidence,Label]
             # (used for majority voting)
    for index, row in df.iterrows():
        for sentence in sent_tokenize(row['Evidence']):
            ev_list.append([row['Claim'],sentence,row['Label']])
            ids.append(row['ID'])
    return(ev_list), ids

triple_to_ids = [0,0,0] #training,validation,testing
cols = ['Claim', 'Evidence', 'Label']
ev_list, triple_to_ids[0] = xtr_evidence(train_df)
xtrain_df = pd.DataFrame(ev_list,columns=cols)
ytrain_df = xtrain_df["Label"]
xtrain_df = xtrain_df.drop("Label",axis=1)

ev_list, triple_to_ids[1] = xtr_evidence(val_df)
xval_df = pd.DataFrame(ev_list,columns=cols)
yval_df = xval_df["Label"]
xval_df = xval_df.drop("Label",axis=1)

ev_list, triple_to_ids[2] = xtr_evidence(test_df)
xtest_df = pd.DataFrame(ev_list,columns=cols)
ytest_df = xtest_df["Label"]
xtest_df = xtest_df.drop("Label",axis=1)
# An inspection
xtrain_df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,Claim,Evidence
0,chris hemsworth appeared in a perfect getaway.,hemsworth has also appeared in the science fic...
1,chris hemsworth appeared in a perfect getaway.,star trek star trek film a perfect getaway a p...
2,roald dahl is a writer.,roald dahl 13 september 1916 -- 23 november 19...
3,roald dahl is a writer.,fighter pilot fighter pilot
4,roald dahl is a governor.,roald dahl 13 september 1916 -- 23 november 19...


## Tokenizing, vocabulary building, encoding - embedding



In [ ]:
## tokenizing
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
# 1st - sentence to words
from tensorflow.keras.preprocessing.text import text_to_word_sequence

T = Tokenizer()

T.fit_on_texts(pd.concat([xtrain_df['Claim'],xtrain_df['Evidence']]))
T.word_index = {k:v-1 for k,v in T.word_index.items()}
T.index_word = {k-1:v for k,v in T.index_word.items()}
word_index = T.word_index
index_word = T.index_word
word_count = dict(T.word_counts)
word_listing = sorted([*word_count.keys()])

xtrain_df['Claim'] = T.texts_to_sequences(xtrain_df['Claim'])
xtrain_df['Evidence'] = T.texts_to_sequences(xtrain_df['Evidence'])
xtest_df['Claim'] = T.texts_to_sequences(xtest_df['Claim'])
xtest_df['Evidence'] = T.texts_to_sequences(xtest_df['Evidence'])
xval_df['Claim'] = T.texts_to_sequences(xval_df['Claim'])
xval_df['Evidence'] = T.texts_to_sequences(xval_df['Evidence'])

xtest_df.head()

,Claim,Evidence
0,"[3474, 16, 104, 9943, 18, 2006, 3249]","[13437, 3474, 16, 104, 9943, 18, 2006, 3249, 2..."
1,"[3474, 16, 104, 9943, 18, 2006, 3249]","[19861, 3968]"
2,"[26441, 5, 9, 4050]","[15965, 26441, 60, 1208, 26441, 5, 3, 1293, 4,..."
3,"[26441, 5, 9, 4050]","[15965, 1508, 15965, 1508, 15965, 902, 15965, ..."
4,"[8684, 98, 4469, 145, 126, 2111, 14861, 5980, ...","[2, 114, 4469, 135, 39, 0, 455, 227, 31587, 52..."


In [ ]:
print(word_index)

{'the': 0, 'of': 1, 'in': 2, 'a': 3, 'and': 4, 'is': 5, 'film': 6, 'was': 7, 'by': 8, 'an': 9, 'for': 10, 'american': 11, 'to': 12, 'on': 13, 'as': 14, 'award': 15, 'has': 16, 'united': 17, 'with': 18, 'born': 19, 'series': 20, 'states': 21, 'best': 22, 'from': 23, 'television': 24, 'actor': 25, 'drama': 26, 'his': 27, "'s": 28, 'he': 29, 'comedy': 30, 'academy': 31, 'world': 32, 'one': 33, 'album': 34, 'directed': 35, 'actress': 36, 'awards': 37, 'known': 38, 'at': 39, 'music': 40, "''": 41, 'first': 42, 'released': 43, 'john': 44, 'it': 45, 'new': 46, 'her': 47, 'rock': 48, 'that': 49, 'city': 50, 'who': 51, 'name': 52, 'which': 53, 'producer': 54, 'films': 55, 'stars': 56, 'written': 57, 'not': 58, 'also': 59, 'or': 60, 'director': 61, 'she': 62, 'fiction': 63, 'tv': 64, 'british': 65, 'won': 66, 'david': 67, 'singer': 68, 'role': 69, 'list': 70, 'golden': 71, 'character': 72, 'only': 73, 'band': 74, 'state': 75, 'science': 76, 'records': 77, 'are': 78, 'game': 79, 'novel': 80, 'mos

In [ ]:
import gensim
import gensim.downloader as gloader

def load_embedding_model(embedding_dimension=50):
    
    emb_model = gloader.load("glove-wiki-gigaword-{}".format(embedding_dimension))

    return emb_model


embedding_dimension = 50

embedding_model = load_embedding_model(embedding_dimension)

[==================================================] 100.0% 66.0/66.0MB downloaded


In [ ]:
# Function definition

def check_OOV_terms(embedding_model, word_listing):

    return list(set(word_listing) - set(embedding_model.vocab.keys()))


oov_terms = check_OOV_terms(embedding_model, word_listing)

print("Total OOV terms: {0} ({1:.2f}%)".format(len(oov_terms), float(len(oov_terms)) / len(word_listing)))

Total OOV terms: 4028 (0.12%)


In [ ]:
import random
import scipy.sparse

def build_embedding_matrix(embedding_model, embedding_dimension, word_index, vocab_size, oov_terms):
    
    embedding_matrix = np.zeros((vocab_size,embedding_dimension),dtype=np.float32)
    
    for w,i in word_index.items():
        if w not in oov_terms:
            embedding_matrix[i] = embedding_model[w]
        else:
            embedding_matrix[i] = np.random.uniform(low=-0.05, high=0.05, size=embedding_dimension)
    
    return embedding_matrix

embedding_matrix = build_embedding_matrix(embedding_model, embedding_dimension, word_index, len(word_index), oov_terms)

print("Embedding matrix shape: {}".format(embedding_matrix.shape))

Embedding matrix shape: (33806, 50)


# Padding and conversion

In [ ]:
from nltk.util import pad_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

lengths = []

lengths.append(len(max(xtrain_df['Evidence'], key=len)))
lengths.append(len(max(xtrain_df['Claim'], key=len)))
lengths.append(len(max(xtest_df['Evidence'], key=len)))
lengths.append(len(max(xtest_df['Claim'], key=len)))
lengths.append(len(max(xval_df['Evidence'], key=len)))
lengths.append(len(max(xval_df['Claim'], key=len)))

m = max(lengths)

xtrain_df['Claim'] = pd.DataFrame([[i] for i in pad_sequences(xtrain_df['Claim'].to_numpy(),m)])
xtrain_df['Evidence'] = pd.DataFrame([[i] for i in pad_sequences(xtrain_df['Evidence'].to_numpy(),m)])
xval_df['Claim'] = pd.DataFrame([[i] for i in pad_sequences(xval_df['Claim'].to_numpy(),m)])
xval_df['Evidence'] = pd.DataFrame([[i] for i in pad_sequences(xval_df['Evidence'].to_numpy(),m)])
xtest_df['Claim'] = pd.DataFrame([[i] for i in pad_sequences(xtest_df['Claim'].to_numpy(),m)])
xtest_df['Evidence'] = pd.DataFrame([[i] for i in pad_sequences(xtest_df['Evidence'].to_numpy(),m)])

print(xtrain_df.shape)


(234126, 2)


In [ ]:
xtrain_df.head()

,Claim,Evidence
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
small_xtrain = xtrain_df[:5000]
small_ytrain = ytrain_df[:5000]

small_xval = xval_df[:500]
small_yval = yval_df[:500]

small_xtest = xtest_df[:1000]
small_ytest = ytest_df[:1000]

In [ ]:
def convert_rows(data):
  total_list = []
  for i in range(len(data)):
      l = []
      for j in range(2):
        l.append(list(data[i,j]))
      total_list.append(l)

  return np.array(total_list)

FAST = False

if FAST:
  small_np_xtrain = convert_rows(small_xtrain.to_numpy())
  small_np_xval = convert_rows(small_xval.to_numpy())
  small_np_xtest = convert_rows(small_xtest.to_numpy())

  small_np_ytrain = np.array(small_ytrain)
  small_np_yval = np.array(small_yval)
  small_np_ytest = np.array(small_ytest)
else:
  np_xtrain = convert_rows(xtrain_df.to_numpy())
  np_xval = convert_rows(xval_df.to_numpy())
  np_xtest = convert_rows(xtest_df.to_numpy())

  np_ytrain = np.array(ytrain_df)
  np_yval = np.array(yval_df)
  np_ytest = np.array(ytest_df)

# Model implementations

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model

input_claim = layers.Input(shape=(m))
input_evidence = layers.Input(shape=(m))

embed_params = {
    "output_dim": 50,
    "input_dim": len(word_index),
    "input_length": m,
    "weights": embedding_matrix if embedding_matrix is None else [embedding_matrix],
    "mask_zero": True,
    "name": "embedding_layer"
}

embed = layers.Embedding(**{key: value for key, value in embed_params.items()})

input_claim_embedded = embed(input_claim)
input_evidence_embedded = embed(input_evidence)

In [ ]:
# 1) Last state as sentence embedding, inputs are concatenated

claim_sentence_embedding = layers.SimpleRNN(50)(input_claim_embedded)
evidence_sentence_embedding = layers.SimpleRNN(50)(input_evidence_embedded)

out = layers.Concatenate()([claim_sentence_embedding,evidence_sentence_embedding])

out = layers.Dense(2,activation="softmax")(out)

model = Model([input_claim,input_evidence],out)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 122)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 122)]        0           []                               
                                                                                                  
 embedding_layer (Embedding)    (None, 122, 50)      1690300     ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 simple_rnn (SimpleRNN)         (None, 50)           5050        ['embedding_layer[0][0]']    

In [ ]:
fit_info = {
    "verbose":True,
    "epochs":5,
    "batch_size":64
}

compile_info = {
    'optimizer': keras.optimizers.Adam(learning_rate=1e-3),
    'loss': 'sparse_categorical_crossentropy',
    'metrics': [keras.metrics.SparseCategoricalAccuracy()],
}

def fit_model(model):
  model.compile(**compile_info)

  if FAST:
    history = model.fit([small_np_xtrain[:,0,:],small_np_xtrain[:,1,:]],small_np_ytrain,
                    validation_data=([small_np_xval[:,0,:],small_np_xval[:,1,:]],small_np_yval),
                    **fit_info)
  else:
    history = model.fit([np_xtrain[:,0,:],np_xtrain[:,1,:]],np_ytrain,
                    validation_data=([np_xval[:,0,:],np_xval[:,1,:]],np_yval),
                    **fit_info)
  return history

In [ ]:
history = fit_model(model)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score
from functools import partial

def get_metric_info(predictions,y,metrics,metric_names):
  assert len(metrics) == len(metric_names)

  metric_info = {}

  for metric, metric_name in zip(metrics, metric_names):
    metric_value = metric(y_pred=predictions, y_true=y)
    metric_info[metric_name] = metric_value

  return metric_info

def evaluate_predictions(test_predictions,y,metrics=None,metric_names=None):
  if metrics is None:
    metrics = [
        accuracy_score,
        partial(f1_score, pos_label=1, average='binary')
    ]
    metric_names = [
        "accuracy",
        "binary_f1"
    ]

  metric_info = get_metric_info(predictions=test_predictions,
                                     y=y,
                                     metrics=metrics,
                                     metric_names=metric_names)

  print('Metrics info: \n{}'.format(metric_info))

  return metric_info

def evaluate(model, metrics = None, metric_names = None):
  assert ((metrics == None) and (metrics == metric_names)) or (len(metrics) == len(metric_names))

  if FAST:
    x = small_np_xtest
    y = small_np_ytest
  else:
    x = np_xtest
    y = np_ytest
  
  predictions = model.predict([x[:,0,:],x[:,1,:]], verbose=True, batch_size=64)

  test_predictions = np.argmax(predictions, axis=-1)

  evaluate_predictions(test_predictions,y,metrics,metric_names)

  return test_predictions

In [ ]:
evaluate(model)

In [ ]:
# 2) Mean of all output states, inputs are concatenated

class AverageSentences(layers.Layer):

    def call(self, inputs, mask=None, training=False, **kwargs):
        """
        float_mask = tf.cast(mask, inputs.dtype)
        masked_inputs = inputs * float_mask[:, :, None]
        return tf.reduce_sum(masked_inputs, axis=1) / tf.reduce_sum(float_mask, axis=1)[:, None]
        """
        assert len(inputs.shape) == 3
        float_mask = tf.cast(mask, inputs.dtype)
        masked_inputs = inputs * float_mask[:, :, None]
        return tf.reduce_sum(masked_inputs, axis=1) / inputs.shape[1]

sequence_output1, claim_sentence_embedding = layers.SimpleRNN(50, return_sequences=True,return_state=True)(input_claim_embedded)
sequence_output2, evidence_sentence_embedding = layers.SimpleRNN(50, return_sequences=True,return_state=True)(input_evidence_embedded)

out1 = AverageSentences()(sequence_output1)
out2 = AverageSentences()(sequence_output2)

out = layers.Concatenate()([out1,out2])

out = layers.Dense(2,activation="softmax")(out)

model = Model([input_claim,input_evidence],out)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 122)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 122)]        0           []                               
                                                                                                  
 embedding_layer (Embedding)    (None, 122, 50)      1690300     ['input_3[0][0]',                
                                                                  'input_4[0][0]']                
                                                                                                  
 simple_rnn_2 (SimpleRNN)       [(None, 122, 50),    5050        ['embedding_layer[0][0]']  

In [ ]:
history = fit_model(model)
evaluate(model)

Epoch 1/2
3659/3659 [==============================] - 501s 136ms/step - loss: 0.4161 - sparse_categorical_accuracy: 0.8209 - val_loss: 0.5943 - val_sparse_categorical_accuracy: 0.7005
Epoch 2/2
214/214 [==============================] - 7s 30ms/step
Test prediction
[1 1 1 ... 1 1 1]
Evaluating predictions! Total samples:  13633
Metrics info: 
{'accuracy': 0.7078412675126531, 'binary_f1': 0.7515748768165658}


In [ ]:
# 3) MLP with reshaping sentence embedding, inputs are concatenated

out1 = layers.Reshape((m*embedding_dimension,))(input_claim_embedded)
out2 = layers.Reshape((m*embedding_dimension,))(input_evidence_embedded)

out1 = layers.Dense(256)(out1)
out1 = layers.Dense(128)(out1)
out1 = layers.Dense(50)(out1)

out2 = layers.Dense(256)(out2)
out2 = layers.Dense(128)(out2)
out2 = layers.Dense(50)(out2)

out = layers.Concatenate()([out1,out2])

out = layers.Dense(2,activation="softmax")(out)

model = Model([input_claim,input_evidence],out)

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 122)]        0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 122)]        0           []                               
                                                                                                  
 embedding_layer (Embedding)    (None, 122, 50)      1690300     ['input_5[0][0]',                
                                                                  'input_6[0][0]']                
                                                                                                  
 reshape (Reshape)              (None, 6100)         0           ['embedding_layer[0][0]']  

In [ ]:
history = fit_model(model)
evaluate(model)

Epoch 1/2
3659/3659 [==============================] - 243s 66ms/step - loss: 0.8546 - sparse_categorical_accuracy: 0.8112 - val_loss: 0.6277 - val_sparse_categorical_accuracy: 0.7144
Epoch 2/2
214/214 [==============================] - 3s 13ms/step
Test prediction
[1 1 1 ... 1 1 1]
Evaluating predictions! Total samples:  13633
Metrics info: 
{'accuracy': 0.6907503850949901, 'binary_f1': 0.7513564519933945}


In [ ]:
# 4) Bag of vectors, inputs are concatenated

out1 = AverageSentences()(input_claim_embedded)
out2 = AverageSentences()(input_evidence_embedded)

out = layers.Concatenate()([out1,out2])

out = layers.Dense(2,activation="softmax")(out)

model = Model([input_claim,input_evidence],out)

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 122)]        0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 122)]        0           []                               
                                                                                                  
 embedding_layer (Embedding)    (None, 122, 50)      1690300     ['input_7[0][0]',                
                                                                  'input_8[0][0]']                
                                                                                                  
 average_sentences_2 (AverageSe  (None, 50)          0           ['embedding_layer[0][0]']  

In [ ]:
history = fit_model(model)
evaluate(model)

Epoch 1/2
3659/3659 [==============================] - 93s 25ms/step - loss: 0.4533 - sparse_categorical_accuracy: 0.7952 - val_loss: 0.6052 - val_sparse_categorical_accuracy: 0.6766
Epoch 2/2
214/214 [==============================] - 1s 4ms/step
Test prediction
[1 1 1 ... 1 1 1]
Evaluating predictions! Total samples:  13633
Metrics info: 
{'accuracy': 0.6896501136947114, 'binary_f1': 0.7511030060591799}


# Merging

We will now show the models without concatenation of the claim and evidence. We will test how the first model (RNN with last output as sentence embedding) performs with the sum and mean of the 2 sentences.

In [ ]:
# 1) Last state as sentence embedding, inputs are summed

claim_sentence_embedding = layers.SimpleRNN(50)(input_claim_embedded)
evidence_sentence_embedding = layers.SimpleRNN(50)(input_evidence_embedded)

out = layers.Add()([claim_sentence_embedding,evidence_sentence_embedding])

out = layers.Dense(2,activation="softmax")(out)

model = Model([input_claim,input_evidence],out)

model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 122)]        0           []                               
                                                                                                  
 input_10 (InputLayer)          [(None, 122)]        0           []                               
                                                                                                  
 embedding_layer (Embedding)    (None, 122, 50)      1690300     ['input_9[0][0]',                
                                                                  'input_10[0][0]']               
                                                                                                  
 simple_rnn_4 (SimpleRNN)       (None, 50)           5050        ['embedding_layer[0][0]']  

In [ ]:
history = fit_model(model)
evaluate(model)

Epoch 1/2
3659/3659 [==============================] - 629s 171ms/step - loss: 0.3842 - sparse_categorical_accuracy: 0.8293 - val_loss: 0.6156 - val_sparse_categorical_accuracy: 0.7138
Epoch 2/2
214/214 [==============================] - 6s 28ms/step
Test prediction
[1 1 1 ... 1 1 1]
Evaluating predictions! Total samples:  13633
Metrics info: 
{'accuracy': 0.7205310643292012, 'binary_f1': 0.7571701720841301}


In [ ]:
# 2) Last state as sentence embedding, inputs are averaged

claim_sentence_embedding = layers.SimpleRNN(50)(input_claim_embedded)
evidence_sentence_embedding = layers.SimpleRNN(50)(input_evidence_embedded)

out = layers.Average()([claim_sentence_embedding,evidence_sentence_embedding])

out = layers.Dense(2,activation="softmax")(out)

model = Model([input_claim,input_evidence],out)

model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 122)]        0           []                               
                                                                                                  
 input_12 (InputLayer)          [(None, 122)]        0           []                               
                                                                                                  
 embedding_layer (Embedding)    (None, 122, 50)      1690300     ['input_11[0][0]',               
                                                                  'input_12[0][0]']               
                                                                                                  
 simple_rnn_6 (SimpleRNN)       (None, 50)           5050        ['embedding_layer[0][0]']  

In [ ]:
history = fit_model(model)
predictions = evaluate(model)

Epoch 1/2
3659/3659 [==============================] - 564s 154ms/step - loss: 0.1107 - sparse_categorical_accuracy: 0.9443 - val_loss: 1.0271 - val_sparse_categorical_accuracy: 0.7197
Epoch 2/2
214/214 [==============================] - 7s 32ms/step
Metrics info: 
{'accuracy': 0.7088681874862466, 'binary_f1': 0.7356291214280956}


In [ ]:
# RNN with averaging of embeddings + cosine similarity

from keras import backend as K

def cosine_distance(vests):
    x, y = vests
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)

def cos_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0],1)

claim_sentence_embedding = layers.SimpleRNN(50)(input_claim_embedded)
evidence_sentence_embedding = layers.SimpleRNN(50)(input_evidence_embedded)

out = layers.Average()([claim_sentence_embedding,evidence_sentence_embedding])

cos_sim = layers.Lambda(cosine_distance, output_shape=cos_dist_output_shape)([claim_sentence_embedding, evidence_sentence_embedding])

out = layers.Concatenate()([out,cos_sim])

out = layers.Dense(2,activation="softmax")(out)

model = Model([input_claim,input_evidence],out)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 122)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 122)]        0           []                               
                                                                                                  
 embedding_layer (Embedding)    (None, 122, 50)      1690300     ['input_3[0][0]',                
                                                                  'input_4[0][0]']                
                                                                                                  
 simple_rnn_2 (SimpleRNN)       (None, 50)           5050        ['embedding_layer[0][0]']  

In [ ]:
history = fit_model(model)
predictions = evaluate(model)

Epoch 1/5
3659/3659 [==============================] - 668s 181ms/step - loss: 0.4019 - sparse_categorical_accuracy: 0.8279 - val_loss: 0.5815 - val_sparse_categorical_accuracy: 0.7215
Epoch 2/5
1982/3659 [===============>..............] - ETA: 3:44 - loss: 0.3006 - sparse_categorical_accuracy: 0.8753

In [ ]:
# RNN with summing of embeddings + cosine similarity

claim_sentence_embedding = layers.SimpleRNN(50)(input_claim_embedded)
evidence_sentence_embedding = layers.SimpleRNN(50)(input_evidence_embedded)

out = layers.Add()([claim_sentence_embedding,evidence_sentence_embedding])

cos_sim = layers.Lambda(cosine_distance, output_shape=cos_dist_output_shape)([claim_sentence_embedding, evidence_sentence_embedding])

out = layers.Concatenate()([out,cos_sim])

out = layers.Dense(2,activation="softmax")(out)

model = Model([input_claim,input_evidence],out)

model.summary()

In [ ]:
history = fit_model(model)
predictions = evaluate(model)

In [ ]:
claim_sentence_embedding = layers.SimpleRNN(50)(input_claim_embedded)
evidence_sentence_embedding = layers.SimpleRNN(50)(input_evidence_embedded)

out = layers.Concatenate()([claim_sentence_embedding,evidence_sentence_embedding])

cos_sim = layers.Lambda(cosine_distance, output_shape=cos_dist_output_shape)([claim_sentence_embedding, evidence_sentence_embedding])

out = layers.Concatenate()([out,cos_sim])

out = layers.Dense(2,activation="softmax")(out)

model = Model([input_claim,input_evidence],out)

model.summary()

#Performance evaluation

We have evaluated each model on the standard evaluation metrics (accuracy, f1 score etc.).<br>
<b>Remember</b>: at the beginning, with <b>xtr_evidence</b>, we splitted each Evidence element based on <b>sent_tokenize</b>. Now, for each (Claim,Evidence1,Evidence2...,Label) tuple, let's try to apply a majority voting, i.e. change the test prediction and see how the value of the metrics change.<br>
We will use the test prediction taken from the RNN model with last output as sentence embedding + averaging the 2 embeddings

In [ ]:
# take max and min ID

min_id = min(triple_to_ids[2])
max_id = max(triple_to_ids[2])

print("Minimum id:", min_id)
print("Maximum id:", max_id)
print(len(triple_to_ids[2]))

Minimum id: 6
Maximum id: 228694
13633


In [ ]:
"""
id_to_prediction:
- keys: ID column from dataset
- value: array of positions of triples having the same ID
"""

id_to_prediction = dict.fromkeys([str(x) for x in triple_to_ids[2]],[])

for i in range(len(triple_to_ids[2])):
  id_to_prediction[str(triple_to_ids[2][i])] = id_to_prediction[str(triple_to_ids[2][i])] + [i]


Streaming output truncated to the last 5000 lines.
158012 [8634]
158012 [8634, 8635]
43326 [8636]
43326 [8636, 8637]
146485 [8638]
146485 [8638, 8639]
76096 [8640]
76096 [8640, 8641]
158017 [8642]
158017 [8642, 8643]
26947 [8644]
26947 [8644, 8645]
26948 [8646]
26948 [8646, 8647]
10567 [8648]
92489 [8649]
92489 [8649, 8650]
92489 [8649, 8650, 8651]
92489 [8649, 8650, 8651, 8652]
92498 [8653]
92498 [8653, 8654]
26964 [8655]
26964 [8655, 8656]
76117 [8657]
76117 [8657, 8658]
76118 [8659]
76118 [8659, 8660]
92507 [8661]
76126 [8662]
76126 [8662, 8663]
76129 [8664]
76129 [8664, 8665]
76129 [8664, 8665, 8666]
76129 [8664, 8665, 8666, 8667]
125292 [8668]
125292 [8668, 8669]
56381 [8670]
56381 [8670, 8671]
141683 [8672]
141683 [8672, 8673]
108919 [8674]
108919 [8674, 8675]
108920 [8676]
108920 [8676, 8677]
125309 [8678]
125309 [8678, 8679]
125309 [8678, 8679, 8680]
125309 [8678, 8679, 8680, 8681]
125309 [8678, 8679, 8680, 8681, 8682]
125315 [8683]
125315 [8683, 8684]
145474 [8685]
145474 [868

In [ ]:
#majority voting

for k,v in id_to_prediction.items():
  supports, refutes = 0, 0
  supports_pos, refutes_pos = [], []
  for j in range(len(v)):
    if predictions[v[j]]:
      supports += 1
      supports_pos.append(v[j])
    else:
      refutes += 1
      refutes_pos.append(v[j])
  if supports > refutes:
    for el in refutes_pos:
      predictions[el] = 1
  else:
    for el in supports_pos:
      predictions[el] = 0

evaluate_predictions(predictions,np_ytest)

Streaming output truncated to the last 5000 lines.
8635
8636
8637
8638
8639
8640
8641
8642
8643
8644
8645
8646
8647
8648
8649
8650
8651
8652
8653
8654
8655
8656
8657
8658
8659
8660
8661
8662
8663
8664
8665
8666
8667
8668
8669
8670
8671
8672
8673
8674
8675
8676
8677
8678
8679
8680
8681
8682
8683
8684
8685
8686
8687
8688
8689
8690
8691
8692
8693
8694
8695
8696
8697
8698
8699
8700
8701
8702
8703
8704
8705
8706
8707
8708
8709
8710
8711
8712
8713
8714
8715
8716
8717
8718
8719
8720
8721
8722
8723
8724
8725
8726
8727
8728
8729
8730
8731
8732
8733
8734
8735
8736
8737
8738
8739
8740
8741
8742
8743
8744
8745
8746
8747
8748
8749
8750
8751
8752
8753
8754
8755
8756
8757
8758
8759
8760
8761
8762
8763
8764
8765
8766
8767
8768
8769
8770
8771
8772
8773
8774
8775
8776
8777
8778
8779
8780
8781
8782
8783
8784
8785
8786
8787
8788
8789
8790
8791
8792
8793
8794
8795
8796
8797
8798
8799
8800
8801
8802
8803
8804
8805
8806
8807
8808
8809
8810
8811
8812
8813
8814
8815
8816
8817
8818
8819
8820
8821
8822
8823
8824

{'accuracy': 0.7107753245800631, 'binary_f1': 0.7353158354031013}

## 4.1 Schema

The overall binary classification problem is summed up by the following (simplified) schema

![](https://drive.google.com/uc?export=view&id=1Wm_YBnFwgJtxcWEBpPbTBEVkpKaL08Jp)

Don't worry too much about the **Encoding** block for now. We'll give you some simple guidelines about its definition. For the moment, stick to the binary classification task definition where, in this case, we have 2 inputs: the claim to verify and one of its associated evidences.

# Architecture Guidelines

There are many neural architectures that follow the above schema. To avoid phenomena like the writer's block, in this section we are going to give you some implementation guidelines.

In particular, we would like you to test some implementations so that you explore basic approaches (neural baselines) and use them as building blocks for possible extensions.

## 5.1 Handling multiple inputs

The first thing to notice is that we are in a multi-input scenario. In particular, each sample is comprised of a fact and its asssociated evidence statement.

Each of these input is encoded as a sequence of tokens. In particular, we will have the following input matrices:

*    Claim: `[batch_size, max_tokens]`
*    Evidence: `[batch_size, max_tokens]`

Moreover, after the embedding layer, we'll have:

*    Claim: `[batch_size, max_tokens, embedding_dim]`
*    Evidence: `[batch_size, max_tokens, embedding_dim]`

But, we would like to have a 2D input to our classifier, since we have to give an answer at pair level. Therefore, for each sample, we would expect the following input shape to our classification block:

*   Classification input shape: `[batch_size, dim]`

**How to do that?**

We inherently need to reduce the token sequence to a single representation. This operation is formally known as **sentence embedding**. Indeed, we are trying to compress the information of a whole sequence into a single embedding vector.

Here are some simple solutions that we ask you to try out:

1.   Encode token sequences via a RNN and take the last state as the sentence embedding.

2.  Encode token sequences via a RNN and average all the output states.

3.  Encode token sequences via a simple MLP layer. In particular, if your input is a `[batch_size, max_tokens, embedding_dim]` tensor, the matrix multiplication works on the **max_tokens** dimension, resulting in a `[batch_size, embedding_dim]` 2D matrix. Alternatively, you can reshape the 3D input tensor from `[batch_size, max_tokens, embedding_dim]` to `[batch_size, max_tokens * embedding_dim]` and then apply the MLP layer.

4.   Compute the sentence embedding as the mean of its token embeddings (**bag of vectors**).

## 5.2 Merging multi-inputs

At this point, we have to think about **how** we should merge evidence and claim sentence embeddings.

For simplicity, we stick to simple merging strategies:

*     **Concatenation**: define the classification input as the concatenation of evidence and claim sentence embeddings

*     **Sum**: define the classification input as the sum of evidence and claim sentence embeddings

*     **Mean**: define the classification input as the mean of evidence and claim sentence embeddings

For clarity, if the sentence embedding of a single input has shape `[batch_size, embedding_dim]`, then the classification input has shape:

*     **Concatenation**: `[batch_size, 2 * embedding_dim]`

*     **Sum**: `[batch_size, embedding_dim]`

*     **Mean**: `[batch_size, embedding_dim]`

# A simple extension

Lastly, we ask you to modify previously defined neural architectures by adding an additional feature to the classification input.

We would like to see if some similarity information between the claim to verify and one of its associated evidence might be useful to the classification.

Compute the cosine similarity metric between the two sentence embeddings and concatenate the result to the classification input.

For clarity, since the cosine similarity of two vectors outputs a scalar value, the classification input shape is modified as follows:

*     **Concatenation**: `[batch_size, 2 * embedding_dim + 1]`

*     **Sum**: `[batch_size, embedding_dim + 1]`

*     **Mean**: `[batch_size, embedding_dim + 1]`



# Performance evaluation

Due to our simplifications, obtained results are not directly compatible with a traditional fact checking method that considers the evidence set as a whole.

Thus, we need to consider two types of evaluations.

---

A. **Multi-input classification evaluation**

This type of evaluation is the easiest and concerns computing evaluation metrics, such as accuracy, f1-score, recall and precision, of our pre-processed dataset.

In other words, we assess the performance of chosen classifiers.

---

B. **Claim verification evaluation**

However, if we want to give an answer concerning the claim itself, we need to consider the whole evidence set. 

Intuitively, for a given claim, we consider all its corresponding (claim, evidence) pairs and their corresponding classification outputs. 

At this point, all we need to do is to compute the final predicted claim label via majority voting.

---

Example:

    Claim: c1
    Evidence set: e1, e2, e3
    True label: S

    Pair outputs:
    (c1, e1) -> S (supports)
    (c1, e2) -> S (supports)
    (c1, e3) -> R (refutes)

    Majority voting:
    S -> 2 votes
    R -> 1 vote

    Final label:
    c1 -> S

Lastly, we have to compute classification metrics just like before.

Shortly speaking, implement both strategies for your classification metrics.

# Tips and Extras

## 8.1 Extensions are welcome!

Is this task too easy for you? Are you curious to try out things you have seen during lectures (e.g. attention)? Feel free to try everything you want!

**Don't forget to try neural baselines first!**

## 8.2 Comments and documentation

Remember to properly comment your code (it is not necessary to comment each single line) and don't forget to describe your work!

## 8.3 Organization

We suggest you to divide your work into sections. This allows you to build clean and modular code, as well as easy to read and to debug.

A possible schema:

*   Dataset pre-processing
*   Dataset conversion
*   Model definition
*   Training
*   Evaluation
*   Comments/Summary

# Evaluation

Which are the evaluation criteria on which we'll judge you and your work?

1. Pre-processing: whether you have done some pre-processing or not.
2. Sentence embedding: you should implement all required strategies (with an example and working code for each). That is, we, as evaluators, should be able to test all strategies without writing down new code.
3. Multiple inputs merging strategies: you should implement all required strategies (with an example and working code for each).
4. Similarity extension: you should implement the cosine similarity extension (with an example and working code).
5. Voting strategy: you should implement the majority voting strategy and provide results.
6. Report: when submitting your notebook, you should also attach a small summary report that describes what you have done (provide motivations as well for abitrary steps. For instance, "We've applied L2 regularization since the model was overfitting".

Extras (possible extra points):

1. Any well defined extension is welcome!
2. Well organized and commented code is as important as any other criteria.

# Contact

For any doubt, question, issue or help, you can always contact us at the following email addresses:

Teaching Assistants:

* Andrea Galassi -> a.galassi@unibo.it
* Federico Ruggeri -> federico.ruggeri6@unibo.it

Professor:

* Paolo Torroni -> p.torroni@unibo.it

*Note*: We highly recommend you to check the [course useful material](https://virtuale.unibo.it/pluginfile.php/1036039/mod_resource/content/2/NLP_Course_Useful_Material.pdf) for additional information before contacting us!

# FAQ

---

**Question**: Can I do something text pre-processing?

**Answer:** You have to! If you check text data, the majority of sentences need some cleaning.

---

**Question**: The model architecture schema is not so clear, are we doing end-to-end training?

**Answer**: Exactly! All models can be thought as:

1. Input
2. (word) Embedding
3. Sentence embedding
4. Multiple inputs merging
5. Classification

---

**Question**: Can I extend models by adding more layers?

**Answer**: Feel free to define model architectures as you wish, but remember satisfy our requirements. This assignment should not be thought as a competition to achieve the best performing model: fancy students that want to show off but miss required assignment objectives will be punished!!

---

**Question**: I'm struggling with the implementation. Can you help me?

**Answer**: Yes sure! Contact us and describe your issue. If you are looking for a particular type of operation, you can easily check the documentation of the deep learning framework you are using (google is your friend).

---

**Question**: Can I try other encoding strategies or neural architectures?

**Answer:** Absolutely! Remember to try out recommended neural baselines first and only then proceed with your extensions.

---

**Question**: Do we have to test all possible sentence embedding and input merging combinations?

**Answer**: Absolutely no! Feel free to pick one sentence embedding strategy and try all possible input merging strategies with it! For instance, pick the best performing sentence embedding method and proceed with next steps (extras included). Please, note that you still have to implement all mentioned strategies!

---

**Question**: I'm hitting out of memory error when training my models, do you have any suggestions?

**Answer**: Here are some common workarounds:

1. Try decreasing the mini-batch size
2. Try applying a different padding strategy (if you are applying padding): e.g. use quantiles instead of maximum sequence length
3. Check the efficiency of your custom code implementation (if any)
4. Try to define same length mini-batches to avoid padding (**It should not be necessary here!**)

---

**Question**: I'm hitting CUDNN_STATUS_BAD_PARAM error! What I'm doing wrong?

**Answer**: This error is a little bit tricky since the stack trace is not meaningful at all! This error occurs when the RNN is fed with a sequence of all 0s and pad masking is enabled (e.g. from the embedding layer). Please, check your conversion step, since there might be an error that leads to the encoding of a sentence to all 0s.

---